## Gerando arquivo para NER-QA-NestedClinBr
Deixar sempre o arquivo de test para fazer o teste completo (com o pipeline todo)
1o. -> passar pelo modelo de NER normal (binario), pegar possiveis entidades
2o. -> combinacao de palavras, modelo de span

Descontinuas: treinar duplicando as frases, tirando os termos do meio.. na hora da predicao: se duas entidades estão mito proximas (ou no mesmo chunk), tirar o espaço e ver se são (ou tentar achar uma regra, ex sempre q tem tratamento e uma anotmia perto, usar uma janela.. ex duas palavras de distancia)...

Para tese: testar tbm sem as descontinuas.. pra pode comparar com outros metodos...

v2 - com corpus original Tempclinbr (arq .txt), contendo as quebras de linha.. aqui quebrar cada frase do texto em uma entrada no dicionario.

v3 - split de train em 75% para dev.. conforme feito no TempClinBr.. e tbm tirar os 41 primeiros tokens dos textos: 
'Data de Criação do Documento: 22/04/2014' como codigo Joao

In [1]:
import os
from pathlib import Path
import re
import pickle
import random


In [2]:
def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']

In [3]:
def replaceWhiteSpaces(str):
    return re.sub('\s{2,}',' ',str)

In [4]:
def save_obj(name, obj):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    with open('../obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    try:
        with open('../obj/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)
    except:
        return None


In [5]:
pastasCorpus = [r'corpus\test',r'corpus\train']
#pastasCorpus = [r'corpus\test']
#pastasCorpus = [r'corpus\TESTE']

In [6]:
import re

s = 'Make the World a 2.4Better Place2.0'
pattern = r'([0-9])\.([0-9])'
replacement = r'\1y\2'
html = re.sub(pattern, replacement, s)

print(html)

Make the World a 2y4Better Place2y0


In [7]:
def getDicSentences(pastaCorpus):
    #devePrintar=True
    devePrintar=False
    dic_sentences = {}
    numMaxTokensPorFrase=0 # numero tokens da maior frase
    num=0
    listaEntidades=[]
    frasesComDescontinuas=[]
    for filename in os.listdir(pastaCorpus):
        f = os.path.join(pastaCorpus, filename)
        if os.path.isfile(f):
            fileNameSemExtensao=os.path.splitext(filename)[0]
            if devePrintar:
                print('\n\n--fileName (sem extensao):--', fileNameSemExtensao)
                pass
            extension = os.path.splitext(filename)[1][1:]
            if extension=='ann':
                # tokens da frase
                fileTxt = open(os.path.join(pastaCorpus, fileNameSemExtensao)+'.txt', "r", encoding='utf-8')
                linha=fileTxt.readlines()
                fileTxt.close()
                if devePrintar:
                    print('linha:', linha)
                    pass
                frases=[]
                allFrasesString=''
                numL=0
                for l in linha:
                    numL=numL+1
                    allFrasesString = allFrasesString+l
                    if numL==1: # descarta primeira frase, que é data de criação do doc
                        #print('descartando frase:', l)
                        continue
                    if l.strip() and l.strip()!='\n':
                        #print('l:', l)
                        pattern = r'([0-9])\.([0-9])'
                        replacement = r'\1==\2'
                        novoL = re.sub(pattern, replacement, l.strip())
                        l2 = novoL.split('.') # quebrando frases
                        for l3 in l2:
                            if l3.strip() and l3.strip()!='\n':
                                novaFrase = l3.replace('\n','').replace('==','.').strip()+'.'
                                frases.append(novaFrase)
                #print('frases:', frases)
                # para cada frase
                # para tokenizar nesses tokens
                #print('allFrasesString:', allFrasesString)
                frasesTokens={}
                #numCaracteresTotal=42 # primeira frase ignorada
                numIndiceAnterior=0
                for frase in frases:
                    tokens=[]
                    frase2 = frase.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                    frase2 = replaceWhiteSpaces(frase2)
                    frase2 = frase2.split()
                    for numtoken, token in enumerate(frase2):
                        if devePrintar:
                            print('token:', token)
                            print('numIndiceAnterior:', numIndiceAnterior)
                        if token!='.':
                            numCaracteresTotal = allFrasesString.find(token, numIndiceAnterior, len(allFrasesString))
                        else:
                            numCaracteresTotal=numIndiceAnterior+1
                        #tokens.append([token,numtoken])
                        tokens.append([token,numtoken, numCaracteresTotal])
                        numIndiceAnterior = numCaracteresTotal+len(token)-1
                        if numMaxTokensPorFrase<len(tokens):
                            numMaxTokensPorFrase = len(tokens)
                    frasesTokens[frase]=tokens
                    #linhaTokens=linha.copy()    
                if devePrintar:
                    print('frasesTokens:', frasesTokens)
                # agora, as entidades
                fileAnn = open(f, "r", encoding='utf-8')
                linha=fileAnn.readlines()
                fileAnn.close()

                dicEntidades={}
                for entidade_linha in linha:
                    if ';' not in entidade_linha: # tem descontinua?
                        entidade = entidade_linha.split('\t')
                        tipo_entidade = entidade[1]
                        inicio, fim = tipo_entidade.split()[1:3]
                        tipo_entidade = tipo_entidade.split()[0]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio), int(fim))]=[tipo_entidade, replaceWhiteSpaces(termos_entidade)]
                    else:
                        frasesComDescontinuas.append(num)
                        #print('descontinua, linha: {}, num: {}'.format(linha, num))
                        #print('descontinua, file: {}, num: {}'.format(filename, num))
                        entidade = entidade_linha.split('\t')
                        # ex T10	Problema 244 252;279 306	dispneia aos mdoeardos-leves esforço
                        #Problema 244 252;279 306
                        entidade_temp=entidade[1].split(';')
                        entidade1=entidade_temp[0]
                        tipo_entidade = entidade1
                        inicio1, fim1 = tipo_entidade.split()[1:3]
                        tipo_entidade_string = tipo_entidade.split()[0]
                        # mandar só os termos referentes...
                        tamTermo1=int(fim1)-int(inicio1)
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[:tamTermo1]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        #print('aaaaaaaaaaaaaaaaa')
                        dicEntidades[(int(inicio1), int(fim1))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]
                        #print("(int(inicio1), int(fim1)):", (int(inicio1), int(fim1)))
                        #print("tipo_entidade_string, replaceWhiteSpaces(termos_entidade):", tipo_entidade_string, replaceWhiteSpaces(termos_entidade))
                        
                        entidade2=entidade_temp[1]
                        inicio2, fim2 = entidade2.split()[0:2]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[tamTermo1:len(termos_entidade)]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio2), int(fim2))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]
                        #print("(int(inicio2), int(fim2)):", (int(inicio2), int(fim2)))
                        #print("tipo_entidade_string, replaceWhiteSpaces(termos_entidade):", tipo_entidade_string, replaceWhiteSpaces(termos_entidade))


                #print('frasesTokens:', frasesTokens)
                indicesDic = sorted(dicEntidades.keys(), key = lambda item: item[0])
                listaIndicesJaUsados = []
                #list_students.sort(key = lambda x: x[1])   #index 1 means second element
                #print(indicesDic)
                #print('dicEntidades:', dicEntidades)
                
                for key, value in frasesTokens.items():
                    for i in indicesDic:
                        tipo_entidade,termos_entidade = dicEntidades[i]
                        ##key (i) = indices old a comparar
                        #print('key:', key)
                        #print('value:', value)
                        #print('i:', i)
                        for token in value:
                            #print('token:', token)
                            if i[0]==token[2]:
                                novo_inicio, novo_fim = [token[1],token[1]+len(termos_entidade.split())]
                                novos_indices=[]
                                for k in range(novo_inicio,novo_fim):
                                    novos_indices.append(k)
                                listaEntidades.append([termos_entidade, novos_indices, tipo_entidade])
                                #print("[termos_entidade, novos_indices, tipo_entidade]:", [termos_entidade, novos_indices, tipo_entidade])
                            else:
                                #print('else, i[0]:',i[0])
                                pass
                        
                    if len(value)>0:
                        #print('incluindo:', key)
                        dic_sentences[num]=[value, listaEntidades]
                        listaEntidades=[]
                        num=num+1 

        #print(num)
        #if num>318:
        #    break

        #if num>3:
        #    break
    print('numMaxTokensPorFrase:', numMaxTokensPorFrase)
    return dic_sentences, frasesComDescontinuas
            


In [8]:
dic_sentencesTest, frasesComDescontinuasTest = getDicSentences(pastasCorpus[0])


numMaxTokensPorFrase: 146


In [9]:
dic_sentencesTest[42]

[[['Paciente', 0, 43],
  ['em', 1, 52],
  ['tratamento', 2, 55],
  ['para', 3, 66],
  ['ICC', 4, 71],
  ['diastólica', 5, 75],
  ['há', 6, 86],
  ['cerca', 7, 89],
  ['de', 8, 95],
  ['5', 9, 98],
  ['a', 10, 100],
  ['com', 11, 102],
  ['melhora', 12, 106],
  ['importante', 13, 114],
  ['dos', 14, 125],
  ['sintomas', 15, 129],
  ['após', 16, 138],
  ['início', 17, 143],
  ['do', 18, 150],
  ['tratamento', 19, 153],
  ['.', 20, 163]],
 [['tratamento', [2], 'Tratamento'],
  ['ICC diastólica', [4, 5], 'Problema'],
  ['melhora importante dos sintomas após início do tratamento',
   [12, 13, 14, 15, 16, 17, 18, 19],
   'Problema'],
  ['tratamento', [19], 'Tratamento']]]

In [10]:
dic_sentencesTrain, frasesComDescontinuasTrain = getDicSentences(pastasCorpus[1])


numMaxTokensPorFrase: 192


In [11]:
print('--Treinamento:--', pastasCorpus[1])
dic_sentencesTrain, frasesComDescontinuasTrain = getDicSentences(pastasCorpus[1])
print('len(sentences):', len(dic_sentencesTrain))
print('len(Descontinuas):',len(frasesComDescontinuasTrain))
print('len(frasesComDescontinuas):',len(set(frasesComDescontinuasTrain)))


print('--Teste:--', pastasCorpus[0])
dic_sentencesTest, frasesComDescontinuasTest = getDicSentences(pastasCorpus[0])
print('len(sentences):', len(dic_sentencesTest))
print('len(Descontinuas):',len(frasesComDescontinuasTest))
print('len(frasesComDescontinuas):',len(set(frasesComDescontinuasTest)))


--Treinamento:-- corpus\train
numMaxTokensPorFrase: 192
len(sentences): 1736
len(Descontinuas): 91
len(frasesComDescontinuas): 51
--Teste:-- corpus\test
numMaxTokensPorFrase: 146
len(sentences): 506
len(Descontinuas): 44
len(frasesComDescontinuas): 17


In [12]:
dic_sentencesTest[9]

[[['Abd', 0, 644],
  ['globoso', 1, 648],
  [',', 2, 655],
  ['flacido', 3, 657],
  [',', 4, 664],
  ['indolor', 5, 666],
  ['a', 6, 674],
  ['palpacao', 7, 676],
  [',', 8, 684],
  ['sem', 9, 686],
  ['VCM', 10, 690],
  ['.', 11, 693]],
 [['Abd', [0], 'Anatomia'], ['VCM', [10], 'Problema']]]

In [13]:
dic_sentencesTrain[861]

[[['CONTRAÇÃO', 0, 507],
  ['SEGMENTAR', 1, 517],
  ['DO', 2, 527],
  ['VE', 3, 530],
  ['ALTERADA', 4, 533],
  ['.', 5, 541]],
 [['CONTRAÇÃO SEGMENTAR DO VE ALTERADA', [0, 1, 2, 3, 4], 'Problema'],
  ['VE', [3], 'Anatomia']]]

In [14]:
frasesComDescontinuasTrain[:10]

[3, 15, 30, 30, 30, 30, 55, 55, 65, 65]

In [15]:
# frases com entidades descontinuas
print(len(set(frasesComDescontinuasTrain)))

51


In [16]:
frasesComDescontinuasTest

[0,
 42,
 42,
 42,
 49,
 67,
 67,
 67,
 67,
 67,
 91,
 131,
 147,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 227,
 241,
 241,
 241,
 241,
 241,
 241,
 241,
 263,
 291,
 291,
 291,
 316,
 329,
 367,
 367,
 367,
 367,
 406,
 468]

In [17]:
print(dic_sentencesTest[2])

[[['Comorbidades', 0, 142], [':', 1, 154], ['DM', 2, 156], ['há', 3, 159], ['10', 4, 162], ['anos', 5, 165], ['em', 6, 170], ['uso', 7, 173], ['de', 8, 177], ['metformina', 9, 180], ['850mg', 10, 191], ['3', 11, 197], ['cp', 12, 199], ['/', 13, 201], ['dia', 14, 202], [',', 15, 205], ['acarbose', 16, 207], ['1', 17, 216], ['cp', 18, 218], ['/', 19, 220], ['dia', 20, 221], ['e', 21, 225], ['glicazida', 22, 227], ['60mg', 23, 237], ['2', 24, 242], ['cp', 25, 244], ['/', 26, 246], ['dia', 27, 247], ['e', 28, 251], ['insulina', 29, 253], ['(', 30, 262], ['24', 31, 263], ['-', 32, 266], ['0', 33, 268], ['-', 34, 270], ['24', 35, 272], [')', 36, 274], ['.', 37, 275]], [['Comorbidades', [0], 'Problema'], ['DM', [2], 'Problema'], ['metformina 850mg', [9, 10], 'Tratamento'], ['acarbose', [16], 'Tratamento'], ['glicazida 60mg', [22, 23], 'Tratamento'], ['insulina', [29], 'Tratamento']]]


In [18]:
#save_obj('dic_sentencesTrainDev',dic_sentencesTrain)
porc=0.76
tamanhoTotal = len(dic_sentencesTrain)
tamanhoTrain = int(tamanhoTotal*porc)
print('tamanhoTrain', tamanhoTrain)
tamanhoDev = tamanhoTotal - tamanhoTrain
print('tamanhoDev', tamanhoDev)
dic_sentencesDev_temp = {k: dic_sentencesTrain[k] for k in list(dic_sentencesTrain)[tamanhoTrain:-1]}
dic_sentencesTrain = {k: dic_sentencesTrain[k] for k in list(dic_sentencesTrain)[:tamanhoTrain]}
num=0
dic_sentencesDev = {}
for key, value in dic_sentencesDev_temp.items():
    dic_sentencesDev[num] = value
    num=num+1

print('len(dic_sentencesTrain):', len(dic_sentencesTrain))
print('len(dic_sentencesDev):', len(dic_sentencesDev))
print(dic_sentencesTrain[tamanhoTrain-1])
print(dic_sentencesDev[0])

tamanhoTrain 1319
tamanhoDev 417
len(dic_sentencesTrain): 1319
len(dic_sentencesDev): 416
[[['Paciente', 0, 151], ['relata', 1, 160], ['apenas', 2, 167], ['um', 3, 174], ['episodio', 4, 177], ['no', 5, 186], ['momento', 6, 189], ['de', 7, 197], ['gripe', 8, 200], ['.', 9, 205]], [['gripe', [8], 'Problema']]]
[[['HAS', 0, 207], [',', 1, 210], ['ICC', 2, 212], [',', 3, 215], ['nega', 4, 217], ['DM', 5, 222], ['.', 6, 224]], [['HAS', [0], 'Problema'], ['ICC', [2], 'Problema'], ['DM', [5], 'Problema']]]


In [19]:
dic_sentencesDev[0]

[[['HAS', 0, 207],
  [',', 1, 210],
  ['ICC', 2, 212],
  [',', 3, 215],
  ['nega', 4, 217],
  ['DM', 5, 222],
  ['.', 6, 224]],
 [['HAS', [0], 'Problema'], ['ICC', [2], 'Problema'], ['DM', [5], 'Problema']]]

In [20]:
dic_sentencesDev[1]

[[['carvedilol', 0, 226],
  ['1', 1, 237],
  ['cp', 2, 239],
  ['12', 3, 242],
  ['/', 4, 244],
  ['12', 5, 245],
  [',', 6, 247],
  ['furosemida', 7, 249],
  ['20mg', 8, 260],
  ['2', 9, 265],
  ['cp', 10, 267],
  ['de', 11, 270],
  ['12', 12, 273],
  ['/', 13, 275],
  ['12', 14, 276],
  [',', 15, 278],
  ['sinvastatina', 16, 280],
  ['1cp', 17, 293],
  ['a', 18, 297],
  ['noite', 19, 299],
  [',', 20, 304],
  ['AAS', 21, 306],
  ['100mg', 22, 310],
  ['apos', 23, 316],
  ['almoço', 24, 321],
  ['e', 25, 328],
  ['Omeprazol', 26, 330],
  ['20mg', 27, 340],
  ['1', 28, 345],
  ['xdia', 29, 347],
  ['.', 30, 351]],
 [['carvedilol', [0], 'Tratamento'],
  ['furosemida 20mg', [7, 8], 'Tratamento'],
  ['sinvastatina', [16], 'Tratamento'],
  ['AAS 100mg', [21, 22], 'Tratamento'],
  ['Omeprazol 20mg', [26, 27], 'Tratamento']]]

In [21]:
save_obj('dic_sentencesTrain',dic_sentencesTrain)
save_obj('dic_sentencesDev',dic_sentencesDev)
save_obj('dic_sentencesTest',dic_sentencesTest)

In [34]:
dic_sentencesTrain[0]

[[['Dispneia', 0, 43],
  ['importante', 1, 52],
  ['aos', 2, 63],
  ['esforços', 3, 67],
  ['+', 4, 76],
  ['dor', 5, 78],
  ['tipo', 6, 82],
  ['peso', 7, 87],
  ['no', 8, 92],
  ['peito', 9, 95],
  ['no', 10, 101],
  ['esforço', 11, 104],
  ['.', 12, 111]],
 [['Dispneia importante aos esforços', [0, 1, 2, 3], 'Problema'],
  ['dor tipo peso no peito no esforço', [5, 6, 7, 8, 9, 10, 11], 'Problema'],
  ['peito', [9], 'Anatomia']]]

In [35]:
listaEntidades=getTiposEntidade()

for i in range(len(dic_sentencesTrain)):
    tokens = dic_sentencesTrain[i][0]
    ents = dic_sentencesTrain[i][1]
    if i>5:
        break
    for tipoEntidade in listaEntidades:
        print('tipoEntidade:', tipoEntidade)
        indiceEnts=[]
        for token in tokens:
            print('token:', token)
            indiceToken = token[1]
            tag='O'
            for ent in ents:
                if indiceToken in ent[1] and ent[2]==tipoEntidade:
                    tag = ent[2]
                    break
            print(tag)
                

tipoEntidade: Problema
token: ['Dispneia', 0, 43]
Problema
token: ['importante', 1, 52]
Problema
token: ['aos', 2, 63]
Problema
token: ['esforços', 3, 67]
Problema
token: ['+', 4, 76]
O
token: ['dor', 5, 78]
Problema
token: ['tipo', 6, 82]
Problema
token: ['peso', 7, 87]
Problema
token: ['no', 8, 92]
Problema
token: ['peito', 9, 95]
Problema
token: ['no', 10, 101]
Problema
token: ['esforço', 11, 104]
Problema
token: ['.', 12, 111]
O
tipoEntidade: Teste
token: ['Dispneia', 0, 43]
O
token: ['importante', 1, 52]
O
token: ['aos', 2, 63]
O
token: ['esforços', 3, 67]
O
token: ['+', 4, 76]
O
token: ['dor', 5, 78]
O
token: ['tipo', 6, 82]
O
token: ['peso', 7, 87]
O
token: ['no', 8, 92]
O
token: ['peito', 9, 95]
O
token: ['no', 10, 101]
O
token: ['esforço', 11, 104]
O
token: ['.', 12, 111]
O
tipoEntidade: Tratamento
token: ['Dispneia', 0, 43]
O
token: ['importante', 1, 52]
O
token: ['aos', 2, 63]
O
token: ['esforços', 3, 67]
O
token: ['+', 4, 76]
O
token: ['dor', 5, 78]
O
token: ['tipo', 6, 82]

In [27]:
# gerar arquivo treinamento
path='data_ner_qa'
f_train = open(path+r'\nested_ner_qa_train.conll', 'w', encoding='utf-8')
listaEntidades=getTiposEntidade()

print('Gravando em ', path)
num_entidade_total=0
num_entidade_train=0
num_entidade_dev=0
num_entidade_test=0


for i in range(len(dic_sentencesTrain)):
    tokens = dic_sentencesTrain[i][0]
    ents = dic_sentencesTrain[i][1]
    for tipoEntidade in listaEntidades:
        indiceEnts=[]
        #f_train.write(tipoEntidade+' O\n[SEP]\n')
        f_train.write(tipoEntidade+' <sep>\n')
        for token in tokens:
            #print('token:', token)
            indiceToken = token[1]
            tag='O'
            for ent in ents:
                if indiceToken in ent[1] and ent[2]==tipoEntidade:
                    tag = ent[2]
                    break
            tokenGravar = token[0].replace(' ','')
            tokenGravar = tokenGravar.strip()
            if tag!=tipoEntidade:
                tag='O'
            else:
                tag='ENT'
            f_train.write(tokenGravar+' '+tag+'\n')
            num_entidade_train=num_entidade_train+1
        f_train.write('\n')        
f_train.close()

f_dev = open(path+r'\nested_ner_qa_dev.conll', 'w', encoding='utf-8')
for i in range(len(dic_sentencesDev)):
    tokens = dic_sentencesDev[i][0]
    ents = dic_sentencesDev[i][1]
    for tipoEntidade in listaEntidades:
        indiceEnts=[]
        f_dev.write(tipoEntidade+' <sep>\n')
        for token in tokens:
            #print('token:', token)
            indiceToken = token[1]
            tag='O'
            for ent in ents:
                if indiceToken in ent[1] and ent[2]==tipoEntidade:
                    tag = ent[2]
                    break
            tokenGravar = token[0].replace(' ','')
            tokenGravar = tokenGravar.strip()
            if tag!=tipoEntidade:
                tag='O'
            else:
                tag='ENT'
            f_dev.write(tokenGravar+' '+tag+'\n')
            num_entidade_dev=num_entidade_dev+1
        f_dev.write('\n')
f_dev.close()


f_test = open(path+r'\nested_ner_qa_test.conll', 'w', encoding='utf-8')
for i in range(len(dic_sentencesTest)):
    tokens = dic_sentencesTest[i][0]
    ents = dic_sentencesTest[i][1]
    indiceEnts=[]
    for tipoEntidade in listaEntidades:
        f_test.write(tipoEntidade+' <sep>\n')
        for token in tokens:
            #print('token:', token)
            indiceToken = token[1]
            tag='O'
            for ent in ents:
                if indiceToken in ent[1] and ent[2]==tipoEntidade:
                    tag = ent[2]
                    break
            tokenGravar = token[0].replace(' ','')
            tokenGravar = tokenGravar.strip()
            if tag!=tipoEntidade:
                tag='O'
            else:
                tag='ENT'
            f_test.write(tokenGravar+' '+tag+'\n')
            num_entidade_test=num_entidade_test+1
        f_test.write('\n')
f_test.close()

print('num_entidade_train', num_entidade_train)
print('num_entidade_dev', num_entidade_dev)
print('num_entidade_test:', num_entidade_test)
num_entidade_total=num_entidade_train+num_entidade_dev+num_entidade_test
print('num_entidade_total:', num_entidade_total)

#save_obj('dic_sentencesTrain',dic_sentencesTrain)
#save_obj('dic_sentencesDev',dic_sentencesDev)
#save_obj('dic_sentencesTest',dic_sentencesTest)


Gravando em  data_ner_qa
num_entidade_train 55204
num_entidade_dev 16036
num_entidade_test: 21812
num_entidade_total: 93052
